In [ ]:
%%capture
%%sh
rm -rf /tmp/jars
mkdir /tmp/jars
wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar -O /tmp/jars/hadoop-aws-3.3.1.jar
wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-core/1.11.901/aws-java-sdk-core-1.11.901.jar -O /tmp/jars/aws-java-sdk-core-1.11.901.jar
wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-s3/1.11.901/aws-java-sdk-s3-1.11.901.jar -O /tmp/jars/aws-java-sdk-s3-1.11.901.jar
wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk/1.11.901/aws-java-sdk-1.11.901.jar -O /tmp/jars/aws-java-sdk-1.11.901.jar
wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-dynamodb/1.11.901/aws-java-sdk-dynamodb-1.11.901.jar -O /tmp/jars/aws-java-sdk-dynamodb-1.11.901.jar


In [ ]:
import configparser
import os

from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

In [ ]:
config = configparser.ConfigParser()
config.read('/home/jovyan/.aws/credentials')
credencials = {}
for key, value in config['default'].items():
    credencials[key] = value

spark_jars = []
for root, dirs, files in os.walk("/tmp/jars"):
    for filename in files:
        spark_jars.append(root + '/' + filename)


In [ ]:
spark = SparkSession \
    .builder \
    .config("spark.sql.sources.partitionOverwriteMode", "dynamic") \
    .config("spark.sql.caseSensitive", True) \
    .config("spark.hadoop.fs.s3a.access.key", credencials['aws_access_key_id']) \
    .config("spark.hadoop.fs.s3a.secret.key", credencials['aws_secret_access_key']) \
    .config("spark.hadoop.fs.s3a.endpoint", f"s3-{credencials['region']}.amazonaws.com") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("com.amazonaws.services.s3.enableV4", True) \
    .config("spark.executor.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.sql.repl.eagerEval.truncate", -1,) \
    .config("spark.sql.repl.eagerEval.maxNumRows", 6) \
    .config("spark.jars", ','.join(spark_jars)) \
    .appName("catch_pokemons") \
    .getOrCreate()


In [ ]:
bucket = "<bucket_name>"
prefix = "<object_prefix>"
s3_path = f"s3a://{bucket}/{prefix}"
df = spark.read.parquet(s3_path)

df